In [1]:
# link: https://www.kaggle.com/gogo827jz/fork-of-keras-multilabel-neural-network

In [2]:
import gc
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.3.1


In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [5]:
MIXED_PRECISION = False
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

Accelerated Linear Algebra enabled


In [3]:
train_features = pd.read_csv('../data/train_features.csv')
train_targets = pd.read_csv('../data/train_targets_scored.csv')
test_features = pd.read_csv('../data/test_features.csv')

ss = pd.read_csv('../data/sample_submission.csv')

In [4]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

In [5]:
train.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,24,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,0,72,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,0,48,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,0,48,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,0,72,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [6]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
        
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation='elu')),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation='elu')),


    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.AdamW(lr = 2e-3, weight_decay = 1e-5, clipvalue = 700), 
                  loss='binary_crossentropy', 
                  )
    return model

In [7]:
from typing import Tuple, List, Callable, Any

from sklearn.utils import check_random_state  # type: ignore

### from eli5
def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False,
                  random_state=None):
    rng = check_random_state(random_state)

    if columns_to_shuffle is None:
        columns_to_shuffle = range(X.shape[1])

    if pre_shuffle:
        X_shuffled = X.copy()
        rng.shuffle(X_shuffled)

    X_res = X.copy()
    for columns in tqdm(columns_to_shuffle):
        if pre_shuffle:
            X_res[:, columns] = X_shuffled[:, columns]
        else:
            rng.shuffle(X_res[:, columns])
        yield X_res
        X_res[:, columns] = X[:, columns]



def get_score_importances(
        score_func,  # type: Callable[[Any, Any], float]
        X,
        y,
        n_iter=5,  # type: int
        columns_to_shuffle=None,
        random_state=None
    ):
    rng = check_random_state(random_state)
    base_score = score_func(X, y)
    scores_decreases = []
    for i in range(n_iter):
        scores_shuffled = _get_scores_shufled(
            score_func, X, y, columns_to_shuffle=columns_to_shuffle,
            random_state=rng, base_score=base_score
        )
        scores_decreases.append(scores_shuffled)

    return base_score, scores_decreases



def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None,
                        random_state=None):
    Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
    res = []
    for X_shuffled in Xs:
        res.append(-score_func(X_shuffled, y) + base_score)
    return res

def metric(y_true, y_pred):
    metrics = []
    for i in range(y_pred.shape[1]):
        if y_true[:, i].sum() > 1:
            metrics.append(log_loss(y_true[:, i], y_pred[:, i]))
    return np.mean(metrics)

perm_imp = np.zeros(train.shape[1])
seed = 0
for n, (tr, te) in enumerate(KFold(n_splits=2, random_state=seed, shuffle=True).split(train_targets)):
    print(f'Fold {n}')

    model = create_model(len(list(train.columns)))
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')

    model.fit(train.values[tr],
              train_targets.values[tr],
              validation_data=(train.values[te], train_targets.values[te]),
              epochs=40, batch_size=128,
              callbacks=[reduce_lr_loss], verbose=2
             )
        
    def _score(X, y):
        pred = model.predict(X)
        return metric(y, pred)

    base_score, local_imp = get_score_importances(_score, train.values[te], train_targets.values[te], n_iter=1, random_state=0)
    perm_imp += np.mean(local_imp, axis=0)
    print('')
    break
    
top_feats = np.argwhere(perm_imp < 0).flatten()
print(top_feats)

Fold 0
Epoch 1/40
94/94 - 2s - loss: 0.3696 - val_loss: 0.0426
Epoch 2/40
94/94 - 2s - loss: 0.0300 - val_loss: 0.0231
Epoch 3/40
94/94 - 2s - loss: 0.0219 - val_loss: 0.0198
Epoch 4/40
94/94 - 2s - loss: 0.0200 - val_loss: 0.0183
Epoch 5/40
94/94 - 2s - loss: 0.0190 - val_loss: 0.0181
Epoch 6/40
94/94 - 2s - loss: 0.0182 - val_loss: 0.0175
Epoch 7/40
94/94 - 2s - loss: 0.0177 - val_loss: 0.0171
Epoch 8/40
94/94 - 2s - loss: 0.0172 - val_loss: 0.0169
Epoch 9/40
94/94 - 2s - loss: 0.0168 - val_loss: 0.0167
Epoch 10/40
94/94 - 2s - loss: 0.0165 - val_loss: 0.0167
Epoch 11/40
94/94 - 2s - loss: 0.0166 - val_loss: 0.0164
Epoch 12/40
94/94 - 2s - loss: 0.0161 - val_loss: 0.0163
Epoch 13/40
94/94 - 2s - loss: 0.0160 - val_loss: 0.0165
Epoch 14/40
94/94 - 2s - loss: 0.0156 - val_loss: 0.0161
Epoch 15/40
94/94 - 2s - loss: 0.0155 - val_loss: 0.0161
Epoch 16/40
94/94 - 2s - loss: 0.0151 - val_loss: 0.0160
Epoch 17/40
94/94 - 2s - loss: 0.0148 - val_loss: 0.0159
Epoch 18/40
94/94 - 2s - loss: 0.

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond

...

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond

C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anacond



[]


C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\win10\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2240: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [8]:
perm_imp

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na